### 一、实现基于豆瓣top250图书评论的简单推荐系统（TF-IDF及BM25两种算法实现）


In [ ]:
import os
import csv
import pandas as pd

# 数据清洗
file_path = "doubanbook_top250_comments.txt"
if not os.path.exists(file_path):
    raise FileExistsError(f"can not found file {file_path}")

infos_list=[]
with open(file=file_path,mode='r',encoding='utf-8') as fs:
    csv_datas = csv.DictReader(fs,delimiter="\t")
    for row in csv_datas:
        infos_list.append(row)

print(len(infos_list))

# 将txt转为csv
df = pd.DataFrame(infos_list)
df.to_csv("doubanbook_top250_comments.csv",index=False)


98175


In [17]:
# 处理book评论信息，依赖stopword进行拆分
import pandas as pd
import jieba

csv_file = "doubanbook_top250_comments.csv"
stop_words_path = "stopwords_full.txt"

if not os.path.exists(csv_file):
    raise FileExistsError(f"can not found file {csv_file}")

if not os.path.exists(stop_words_path):
    raise FileExistsError(f"can not found file {stop_words_path}")

df = pd.read_csv(csv_file)
df['body'] = df['body'].fillna('')
book_infos = df.to_dict('records')

book_comments = {}
for book_info in book_infos:
    book = book_info['book']
    if book == '' : continue

    comment = book_info['body']
    # print(comment)
    comment_words = jieba.lcut(comment)

    book_comments[book] = book_comments.get(book,[])
    book_comments[book].extend(comment_words)

stop_words_ = []
with open(file=stop_words_path,mode='r',encoding='utf-8') as sw_fs:
    stop_words_ = [line.strip() for line in sw_fs if line.strip()]

book_names=list(book_comments.keys())
book_cts = list(book_comments.values())


#### TF-IDF

In [52]:
# 计算TF-IDF并通过计算余弦相似度
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 构建TF-IDF特征矩阵
vectorize = TfidfVectorizer(stop_words=stop_words_)
tfidf_matrix = vectorize.fit_transform([' '.join(comments_) for comments_ in book_cts])

# 计算图书之间余弦相似度
similarity_matrix = cosine_similarity(tfidf_matrix)

input_book_name ="盗墓笔记"

if input_book_name not in book_names:
    raise ValueError("input book name not found.")
book_idx = book_names.index(input_book_name)

# 获取与输入图书最相似的图书
recomment_book_indexs = np.argsort(-similarity_matrix[book_idx])[1:11]

# 输出推荐书籍
for idx in recomment_book_indexs:
    print(f"《{book_names[idx]}》 相似度：「{similarity_matrix[book_idx][idx]:.4f}」")




/opt/miniconda3/envs/py311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl', 'daren', 'itse', 'lex', 'mayn', 'mon', 'myse', 'oughtn', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


《鬼吹灯之精绝古城》 相似度：「0.6717」
《鬼吹灯之云南虫谷》 相似度：「0.5673」
《盗墓笔记3》 相似度：「0.5351」
《盗墓笔记4》 相似度：「0.4690」
《盗墓笔记2》 相似度：「0.4114」
《达·芬奇密码》 相似度：「0.2869」
《西决》 相似度：「0.2629」
《何以笙箫默》 相似度：「0.2589」
《华胥引（全二册）》 相似度：「0.2448」
《穆斯林的葬礼》 相似度：「0.2384」


#### BM25

In [51]:
from rank_bm25 import BM25Okapi
import numpy as np

preprocessed_corpus = []  # 存储过滤停用词后的所有书评
for words in book_comments.values():
    filtered_words = [word for word in words if word not in stop_words_ and word.strip() != '']
    preprocessed_corpus.append(filtered_words)

bm25 = BM25Okapi(preprocessed_corpus)  # 初始化 BM25 模型

input_book_name ="盗墓笔记"

if input_book_name not in book_names:
    raise ValueError("input book name not found.")
book_idx = book_names.index(input_book_name)

query = preprocessed_corpus[book_idx]  # 获取这本书的词列表
scores = bm25.get_scores(query)  # 计算相似度分数

sorted_indices = np.argsort(scores)[::-1]  # 按相似度降序排序
recommended_indices = [idx for idx in sorted_indices if idx != book_idx][1:11] 

# 输出推荐书籍
for idx in recomment_book_indexs:
    print(f"《{book_names[idx]}》 相似度：「{similarity_matrix[book_idx][idx]:.4f}」")

《鬼吹灯之精绝古城》 相似度：「0.6717」
《鬼吹灯之云南虫谷》 相似度：「0.5673」
《盗墓笔记3》 相似度：「0.5351」
《盗墓笔记4》 相似度：「0.4690」
《盗墓笔记2》 相似度：「0.4114」
《达·芬奇密码》 相似度：「0.2869」
《西决》 相似度：「0.2629」
《何以笙箫默》 相似度：「0.2589」
《华胥引（全二册）》 相似度：「0.2448」
《穆斯林的葬礼》 相似度：「0.2384」


### 二、使用自定义的文档文本，通过fasttext训练word2vec训练词向量模型，并计算词汇间的相关度。（选做：尝试tensorboard绘制词向量可视化图）

In [53]:
import jieba
import re

data_src_path = 'data_src.txt'
data_path = 'data.txt'

if not os.path.exists(data_src_path):
    raise FileExistsError(f"can not found file {data_src_path}")

texts = []
with open(file=data_src_path,mode='r',encoding='utf-8') as fs:
    for line in fs:
        words = jieba.lcut(re.sub(r'[^\w\s]', '', line.strip()))
        texts.append(' '.join(words))

with open(file=data_path, mode='w', encoding='utf-8') as fs:
    for text in texts:
        fs.write(text + '\n')

#### FastText

In [55]:
import fasttext
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import os

data_path = 'data.txt'
word_pairs = [("科技", "创新"), ("手机", "电脑"), ("艺术", "科学")]

# 无监督学习,使用skipgram（跳字模型）模型
model_skipgram = fasttext.train_unsupervised(
    input=data_path,
    model='skipgram',
    dim=100,ws=5,
    minCount=5,
    epoch=50)

# 使用cbow（连续词袋模型）
# model_cbow = fasttext.train_unsupervised('data.txt',model='cbow',dim=100,ws=5,minCount=5,epoch=50)

model = model_skipgram

model.save_model('fasttext_model.bin')

# 计算词语相似度
for w1, w2 in word_pairs:
    vec1 = model.get_word_vector(w1).reshape(1, -1)
    vec2 = model.get_word_vector(w2).reshape(1, -1)
    sim = cosine_similarity(vec1, vec2)[0][0]
    print(f"{w1} vs {w2}: {sim:.4f}")

words = model.words[:500]
vectors = np.array([model.get_word_vector(w) for w in words])
    
# 创建TF变量
embedding_var = tf.Variable(vectors, name=f'fasttext_{len(words)}d')
    
# 保存模型和metadata
log_dir = 'fasttext_vis'
os.makedirs(log_dir, exist_ok=True)

with open(f'{log_dir}/metadata.tsv', 'w', encoding='utf-8') as f:
    f.write("Word\tFrequency\n")
    for i, word in enumerate(words):
        f.write(f"{word}\t{i+1}\n")  # 用序号模拟词频

# 保存embedding
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(log_dir, 'embedding.ckpt'))

# print(f"\n启动TensorBoard查看:\ntensorboard --logdir {log_dir}\n")
# os.system("tensorboard --logdir fasttext_vis")

Read 0M words
Number of words:  2
Number of labels: 0
Progress: 100.0% words/sec/thread:    6136 lr:  0.000000 avg.loss:  4.108504 ETA:   0h 0m 0s


科技 vs 创新: -0.0538
手机 vs 电脑: 0.0140
艺术 vs 科学: 0.0640


'fasttext_vis/embedding.ckpt-1'

#### word2Vec

In [ ]:
import os
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

data_path = 'data.txt'
word_pairs = [("科技", "创新"), ("手机", "电脑"), ("艺术", "科学")]

# 从文件加载分词后的数据（每行已分词，用空格分隔）
sentences = []
with open(data_path, 'r', encoding='utf-8') as fs:
    sentences = [line.strip().split() for line in fs]

# 训练Word2Vec模型
model = Word2Vec(
    sentences=sentences,
    vector_size=100,    # 向量维度
    window=5,           # 上下文窗口
    min_count=5,        # 最小词频
    sg=1,               # 1=skipgram, 0=CBOW
    epochs=50           # 迭代次数
)

# 保存模型
model.save("word2vec.model")

# 计算词语相似度（与FastText API一致）
for w1, w2 in word_pairs:
    sim = model.wv.similarity(w1, w2)  # 直接调用gensim的相似度方法
    print(f"{w1} vs {w2}: {sim:.4f}")

# 获取词表和向量
words = list(model.wv.index_to_key)[:500]
vectors = np.array([model.wv[word] for word in words])

# 保存模型和metadata
log_dir = 'fasttext_vis'
os.makedirs(log_dir, exist_ok=True)
# 1. 保存词向量为TF变量
embedding_var = tf.Variable(vectors, name='word2vec_embedding')
# 2. 生成Metadata（词+自定义属性）
with open(os.path.join(log_dir, 'metadata.tsv'), 'w', encoding='utf-8') as f:
    f.write("Word\tFrequency\n")
    for i, word in enumerate(words):
        f.write(f"{word}\t{i+1}\n")
# 3. 保存向量和配置
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(log_dir, 'embedding.ckpt'))

print(f"\n启动TensorBoard查看:\ntensorboard --logdir {log_dir}\n")
os.system("tensorboard --logdir fasttext_vis")

### 三、使用课堂示例cooking.stackexchange.txt，使用fasttext训练文本分类模型。（选做：尝试使用Kaggle中的Fake News数据集训练文本分类模型）

In [57]:
import re
import jieba

input_file = "cooking.stackexchange.txt"
output_file = "cooking.fasttext.txt"

if not os.path.exists(input_file):
    raise FileExistsError(f"can not found file {input_file}")

texts = []
with open(file=input_file,mode='r',encoding='utf-8') as fs:
    for line in fs:
        words = jieba.lcut(re.sub(r'[^\w\s]', '', line.strip()))
        texts.append(' '.join(words))

with open(file=output_file, mode='w', encoding='utf-8') as fs:
    for text in texts:
        fs.write(text + '\n')

In [ ]:
import fasttext

# 训练模型（监督学习）
model = fasttext.train_supervised(
    input="cooking.stackexchange.txt",
    lr=0.1, 
    epoch=25,         
    wordNgrams=2,     
    dim=100,          
    loss='softmax' 
)

# 保存模型
model.save_model("cooking_model.bin")

# 加载模型
model = fasttext.load_model("cooking_model.bin")

# 预测单条文本
text = "Which backing dish is best to bake a banana bread ?"
print(model.predict(text))